## Simple UDF sample

This notebook shows how to apply [user defined functions](https://open-eo.github.io/openeo-python-client/udf.html).
This is a powerfull concept that allows you to combine arbitrary Python code with your predefined openEO processes.

We start by setting up a basic data cube:


In [1]:
import openeo


# Create connection to openEO back-end
connection = openeo.connect("openeo.cloud").authenticate_oidc("egi-legacy")


# Load initial data cube.
s2_cube = connection.load_collection(
    "SENTINEL2_L2A",
    spatial_extent={"west": 4.00, "south": 51.04, "east": 4.01, "north": 51.05},
    temporal_extent=["2022-03-01", "2022-03-31"],
    bands=["B02", "B03", "B04"]
)


To authenticate: visit https://aai.egi.eu/oidc/device and enter the user code 'Ffu6O2'.
Authorized successfully.
Authenticated using device code flow.


Now we load the UDF, it needs to be provided to the backend as plain text, so we store it in a separate file next to this notebook. This makes it easier to edit.

In [16]:
from pathlib import Path
udf_code = Path("simple_udf.py").read_text(encoding="utf8")


Now we want to apply the UDF to each pixel. To achieve this, we use a [simple callback](https://open-eo.github.io/openeo-python-client/processes.html#processes-with-child-callbacks).

In [18]:
scaling_process = lambda data: data.run_udf(udf = udf_code, runtime = 'Python')
# Pass UDF object as child process to `apply`.
rescaled = s2_cube.apply(scaling_process)
rescaled.download("apply-udf-scaling.nc")